# 06 Stability mismatch proxy
Hypothesis: `E_proxy = feature_drift + (1 - source_retention_acc)` tracks unstable transfer.

### Expected Outcome
Naive fine-tuning should maintain higher `E_proxy` than gradual unfreezing.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
LAB.run('transfer_safe_vs_naive.yaml', '06_stability', fast_dev_run=FAST_DEV_RUN)
safe = LAB.read_method('gradual_unfreeze').copy()
naive = LAB.read_method('naive_finetune').copy()
for df in (safe, naive):
    df['E_proxy'] = df['feature_drift'] + (1.0 - df['source_retention_acc'])

In [ ]:
fig, ax = plt.subplots(figsize=(6.6, 3.7))
ax.plot(safe['epoch'], safe['E_proxy'], marker='o', label='gradual_unfreeze')
ax.plot(naive['epoch'], naive['E_proxy'], marker='o', label='naive_finetune')
ax.set_title('Mismatch proxy over adaptation')
ax.set_xlabel('epoch')
ax.set_ylabel('E_proxy')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '06_e_proxy_by_method.png')

fig, ax = plt.subplots(figsize=(6.6, 3.7))
ax.plot(safe['epoch'], safe['target_test_acc'], marker='o', label='gradual_unfreeze')
ax.plot(naive['epoch'], naive['target_test_acc'], marker='o', label='naive_finetune')
ax.set_title('Target accuracy under matched setup')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '06_target_acc_safe_vs_naive.png')

In [ ]:
stability = pd.DataFrame({
    'epoch': safe['epoch'],
    'E_safe': safe['E_proxy'],
    'E_naive': naive['E_proxy'],
    'target_safe': safe['target_test_acc'],
    'target_naive': naive['target_test_acc'],
})
stability.tail()

### Usage
If `E_proxy` rises while target gains flatten, adaptation is too aggressive.